###### convnet mnist digit example
import sys;print('python version:',sys.version)
import tensorflow as tf; print('tf_version:',tf.__version__) #tf: 2.1
from tensorflow import keras; print('keras version:',keras.__version__)
from tensorflow.python.platform import build_info as tf_build_info
print('CUDA Version:',tf_build_info.cuda_version_number)
print('CUDNN Version:',tf_build_info.cudnn_version_number)

import keras
print("keras version on its own:",keras.__version__) #tf keras and keras library can have different versions

In [26]:
import sys;print('python version:',sys.version) 
import tensorflow as tf;print('tf_version:',tf.__version__) #tf: 2.1 
from tensorflow import keras; print('keras version:',keras.__version__) 
from tensorflow.python.platform import build_info as tf_build_info 
print('CUDA Version:',tf_build_info.cuda_version_number) 
print('CUDNN Version:',tf_build_info.cudnn_version_number)
import keras 
print("keras version on its own:",keras.__version__) #tf keras and keras library can have different versions

python version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
tf_version: 2.1.0
keras version: 2.2.4-tf
CUDA Version: 10.1
CUDNN Version: 7.6
keras version on its own: 2.3.1


In [4]:
#layers learn features or distill data

from keras import layers
from keras import models

model = models.Sequential() #model type?
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) #layers accept data as tensors. Our image goes in as a (28x28,BW) tensor. Conv learns local featrures on a 3x3 window
model.add(layers.MaxPooling2D((2, 2))) #compress data to half each time it is applied. Applied on a 2.2 window
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#output tensor comes as (3x3x64). It is a 3D tensor still - HxWxChannels. Channels controlled by 1st argument passed to the Conv2D layer

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.add(layers.Flatten()) #below dense layers is a classifier network. but classifiers only accept the tensor in 1D, so we flatten it
model.add(layers.Dense(64, activation='relu')) #wout convnet, we used 512 layers and yet the accuracy was 97.8%
model.add(layers.Dense(10, activation='softmax')) #only 10 outputs

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [8]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) #60k images of 28*28*1
train_images = train_images.astype('float32') / 255 #grayscale normalized from 0-255 to 0-1

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels) #each image input has an associated answer as a label 
test_labels = to_categorical(test_labels)

In [9]:
#Making network ready for training
model.compile(optimizer='rmsprop', #optimizes
              loss='categorical_crossentropy', #gives the direction
              metrics=['accuracy'])
#fitting the data (answers, input images) into the model is the same as training
model.fit(train_images, train_labels, epochs=5, batch_size=64)



Epoch 1/5
60000/60000 [==============================] - 17s 286us/step - loss: 0.1636 - accuracy: 0.9481
Epoch 2/5
60000/60000 [==============================] - 18s 295us/step - loss: 0.0444 - accuracy: 0.9863
Epoch 3/5
60000/60000 [==============================] - 18s 302us/step - loss: 0.0305 - accuracy: 0.9905
Epoch 4/5
60000/60000 [==============================] - 18s 304us/step - loss: 0.0238 - accuracy: 0.9930
Epoch 5/5
60000/60000 [==============================] - 19s 323us/step - loss: 0.0187 - accuracy: 0.9941


In [10]:
#Evaluate model training
train_loss, train_acc = model.evaluate(train_images, train_labels)
train_acc

60000/60000 [==============================] - 6s 101us/step


0.9964500069618225

In [11]:
#Actual Model Evaluation
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc #68% more relative accuracy with convolution at 8 times less dense layers (512vs64)

10000/10000 [==============================] - 1s 100us/step


0.9914000034332275

In [14]:
train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [15]:
test_labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
import tensorflow as tf
tf.test.is_gpu_available() # True/False

# Or only check for gpu's with cuda support
tf.test.is_gpu_available(cuda_only=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [18]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1240154929661912436
]
